In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import  pandas_profiling

In [2]:
# read data for the Kaggle estimation
X_kaggle = pd.read_csv("test.csv")
pass_id = pd.DataFrame(X_kaggle["PassengerId"])


In [3]:
# define X & Y sets
data = pd.read_csv("train.csv")
X = data.drop("Survived", axis = 1)
Y = data["Survived"]

In [4]:
# analyse data
report = data.profile_report()
report.to_file(output_file="ritanic_dataset_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# check empty data
X.isna().sum(), Y.isna().sum()

(PassengerId      0
 Pclass           0
 Name             0
 Sex              0
 Age            177
 SibSp            0
 Parch            0
 Ticket           0
 Fare             0
 Cabin          687
 Embarked         2
 dtype: int64,
 0)

In [6]:
# Check whether we need type tranformation
X["Age"].dtype, type(X["Embarked"][0])

(dtype('float64'), str)

In [7]:
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
# Imputation by pandas
X = X.drop(["Name", "Ticket", "Cabin", "PassengerId"], axis = 1)

men_mean = X[X["Sex"] == 'male']["Age"].mean()
women_mean = X[X["Sex"] == 'female']["Age"].mean()

X["Embarked"] = X["Embarked"].fillna("S")

X['Age'] = np.where((X.Sex == 'female'), women_mean, X.Age)
X['Age'] = np.where((X.Sex == 'male'), men_mean, X.Age)

In [9]:
# the same imputation for the Kaggle estimation data
X_kaggle = X_kaggle.drop(["Name", "Ticket", "Cabin", "PassengerId"], axis = 1)

X_kaggle["Fare"] = X_kaggle["Fare"].fillna(X_kaggle["Fare"].mean())


X_kaggle['Age'] = np.where((X_kaggle.Sex == 'female'), women_mean, X_kaggle.Age)
X_kaggle['Age'] = np.where((X_kaggle.Sex == 'male'), men_mean, X_kaggle.Age)

In [10]:
# check empty data again
X.isna().sum(), X_kaggle.isna().sum()

(Pclass      0
 Sex         0
 Age         0
 SibSp       0
 Parch       0
 Fare        0
 Embarked    0
 dtype: int64,
 Pclass      0
 Sex         0
 Age         0
 SibSp       0
 Parch       0
 Fare        0
 Embarked    0
 dtype: int64)

In [11]:
# change categorical data to numbers
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


categorical_features = ["Pclass", "Sex", "Embarked"]

enc = OneHotEncoder(categories='auto')
transform = ColumnTransformer([("enc", enc, categorical_features)], remainder = 'passthrough')

X = transform.fit_transform(X)
X_kaggle = transform.fit_transform(X_kaggle)

In [12]:
# split X, Y by train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [13]:
#create a model
#begin with RandomForestClassification
from sklearn.ensemble import RandomForestClassifier

np.random.seed(42)
model = RandomForestClassifier(n_estimators = 50)
model.fit(X_train, Y_train)


RandomForestClassifier(n_estimators=50)

In [14]:
# estimate RandomForest
model.score(X_test, Y_test)

0.7932960893854749

In [15]:
#lets try another model SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [16]:
#estimate SVC model

clf.score(X_test, Y_test)

0.8100558659217877

In [17]:
###### let's try K-neighbours classifier
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, Y_train)


KNeighborsClassifier(n_neighbors=3)

In [18]:
#estimate K-neighbours

neigh.score(X_test, Y_test)

0.8156424581005587

In [20]:
# final file creation for the Kaggle estimation

y_preds = clf.predict(X_kaggle)

Y_kaggle = pass_id
Y_kaggle["Survived"] = y_preds
Y_kaggle.to_csv("titanic_predictions.csv", index = False)